In [3]:
import time
import yfinance as yf
from concurrent.futures import ThreadPoolExecutor, as_completed
from earnings_pipline.util import get_all_tickers

In [ ]:
def fetch_meta_yfinance(symbols, max_workers=8, pause=0.0):
    symbols = get_all_tickers()
    out = {}

    def one(sym):
        try:
            info = yf.Ticker(sym.info)
        except Exception:
            info = {}

        return sym, {
            "name": info.get("longName") or info.get("shortName"),
            "sector": info.get("sector"),
            "industry": info.get("industry"),
        }
    
    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        futs = [ex.subit(one, s) for s in symbols]
        for fut in as_completed(futs):
            sym, rec = fut.result()
            out[sym] = rec
            if pause:
                return time.sleep(pause)
    return out